In [ ]:
import os
from collections import namedtuple

import findspark

os.environ['SPARK_HOME'] = '/Users/audioworkstation/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/spark-3.5.0-bin-hadoop3'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'

findspark.init()

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

sc = SparkContext()
ssc = StreamingContext(sc, 20) # Checks every 20 seconds for incoming data
sql_context = SQLContext(sc)

socket_stream = ssc.socketTextStream("127.0.0.1", 5555)
socket_stream.pprint()
# decoded_lines = socket_stream.map(lambda data: data.decode())
# decoded_lines.pprint()


fields = ('tag', 'count')
Tweet = namedtuple('Tweet', fields)

(socket_stream.flatMap(lambda text: text.split(" "))
 .filter(lambda word: word.lower().startswith("#"))
 .map(lambda word: (word.lower(), 1))
 .reduceByKey(lambda a, b: a + b)
 .map(lambda rec: Tweet(rec[0], rec[1]))
 .foreachRDD(lambda rdd: process_rdd(rdd)))

def process_rdd(rdd):
    if not rdd.isEmpty():
        print("Received RDD with data:")
        for record in rdd.collect():
            print(record)
        rdd.toDF().sort(desc("count")).limit(10).registerTempTable("tweets")
    else:
        print("Received empty RDD")




# start socket here
### Unfortunately, my python tweeoy_sampler does not work. 
### I'm creating socket using nc -lk 5555

###### sample socket data
```
A #python B
A #data B
A #data B
A #data B
A #data B
A #engineer B
A #python B
A #engineer B
A #python B
A #python B
A #data B
A #data B
A #data B
A #data B
A #engineer B
A #python B
A #engineer B
A #python B
A #python B
A #data B
A #data B
A #data B
A #data B
A #engineer B
A #python B
A #engineer B
A #python B
A #python B
A #data B
A #data B
A #data B
A #data B
A #engineer B
A #python B
A #engineer B
A #python B
A #python B
A #data B
A #data B
A #data B
A #data B
A #engineer B
A #python B
A #engineer B
A #python B
A #python B
A #data B
A #data B
A #data B
A #data B
A #engineer B
A #python B
A #engineer B
A #python B
A #python B
A #data B
A #data B
A #data B
A #data B
A #engineer B
A #python B
A #engineer B
A #python B
A #python B
A #data B
A #data B
A #data B
A #data B
A #engineer B
A #python B
A #engineer B
A #python B
A #python B
A #data B
A #data B
A #data B
A #data B
A #engineer B
A #python B
A #engineer B
A #python B
A #python B
A #data B
A #data B
A #data B
A #data B
A #engineer B
A #python B
A #engineer B
A #python B
A #python B
A #data B
A #data B
A #data B
A #data B
A #engineer B
A #python B
A #engineer B
A #python B
A #python B
A #data B
A #data B
A #data B
A #data B
A #engineer B
A #python B
A #engineer B
A #python B
A #python B
A #data B
A #data B
A #data B
A #data B
A #engineer B
A #python B
A #engineer B
A #python B
A #python B
A #data B
A #data B
A #data B
A #data B
A #engineer B
A #python B
A #engineer B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
A #python B
```

# ploting

In [ ]:
ssc.start()

In [ ]:
import time
from IPython import display

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
%matplotlib inline

In [ ]:

count = 0
while count < 10:
    time.sleep(3)
    top_10 = sql_context.sql("SELECT tag, count from tweets")
    top_10_pd_df = top_10.toPandas()
    display.clear_output(wait=True)
    plt.figure(figsize= (10, 8))
    sns.barplot(x='count', y='tag', data=top_10_pd_df)
    plt.show()
    count = count + 1

    

In [ ]:
ssc.stop()
sc.stop()